In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from utils import imageUtils
from utils import postProcessing
from utils import DefectDetectionDataset
from utils import evaluation
from utils import visualization
import numpy as np
import os
from chainercv.links import FasterRCNNVGG16
from chainercv.visualizations import vis_bbox
from chainercv.utils import write_image
import chainer
import math
from chainercv import utils
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
#load Data
root = './data/3Types/dataset1024x1024'
dataset = DefectDetectionDataset(data_dir=root, split='train')
#dataset_test = DefectDetectionDataset(data_dir=root, split='test')
bbox_label_names = ('111loop')

FileNotFoundError: [Errno 2] No such file or directory: './data/3Types/dataset1024x1024/trainimages.txt'

In [10]:
# DataSet Statistics
print('total number of training images: ', len(dataset))
print('total number of test images: ', len(dataset))
print('type of defects: ', bbox_label_names)

total number of training images:  213
total number of test images:  213
type of defects:  111loop


## Transform Class

In [13]:
class Transform(object):

    def __init__(self, faster_rcnn):
        self.faster_rcnn = faster_rcnn

    def __call__(self, in_data):
        img, bbox, label = in_data
        _, H, W = img.shape

        # random brightness and contrast
        img = random_distort(img)

        # rotate
        img, params = transforms.random_rotate(img, return_param=True)
        _, t_H, t_W = img.shape
        bbox = rotate_bbox(bbox, (H, W), params['k'])
        
        # Random expansion
        if np.random.randint(2):
            img, param = transforms.random_expand(img, max_ratio=2, return_param=True)
            bbox = transforms.translate_bbox(
                bbox, y_offset=param['y_offset'], x_offset=param['x_offset'])

        # Random crop
        img, param = random_crop_with_bbox_constraints(
            img, bbox, return_param=True)        
        bbox, param = transforms.crop_bbox(
            bbox, y_slice=param['y_slice'], x_slice=param['x_slice'],
            allow_outside_center=False, return_param=True)
        label = label[param['index']]
        
        _, t_H, t_W = img.shape

        img = self.faster_rcnn.prepare(img)
        _, o_H, o_W = img.shape
        bbox = transforms.resize_bbox(bbox, (t_H, t_W), (o_H, o_W))

        # horizontally & vertical flip
        img, params = transforms.random_flip(
            img, x_random=True, y_random=True, return_param=True)
        bbox = transforms.flip_bbox(
            bbox, (o_H, o_W), x_flip=params['x_flip'], y_flip=params['y_flip'])

        scale = o_H / t_H

        return img, bbox, label, scale

In [11]:
# Test data augumeation
print(len(dataset))

213


In [12]:
for i in range(len(dataset)):
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
